In [7]:
from scipy.optimize import linprog
import numpy as np
import math
import sys
from queue import Queue


class ILP:
    def __init__(self, c, A_ub, b_ub, A_eq, b_eq, bounds):
        # 全局参数
        self.LOWER_BOUND = -sys.maxsize
        self.UPPER_BOUND = sys.maxsize
        self.opt_val = None
        self.opt_x = None
        self.Q = Queue()

        # 这些参数在每轮计算中都不会改变
        self.c = -c
        self.A_eq = A_eq
        self.b_eq = b_eq
        self.bounds = bounds

        # 首先计算一下初始问题
        r = linprog(-c, A_ub, b_ub, A_eq, b_eq, bounds)

        # 若最初问题线性不可解
        if not r.success:
            raise ValueError("Not a feasible problem!")

        # 将解和约束参数放入队列
        self.Q.put((r, A_ub, b_ub))

    def solve(self):
        while not self.Q.empty():
            # 取出当前问题
            res, A_ub, b_ub = self.Q.get(block=False)

            # 当前最优值小于总下界，则排除此区域
            if -res.fun < self.LOWER_BOUND:
                continue

            # 若结果 x 中全为整数，则尝试更新全局下界、全局最优值和最优解
            if all(list(map(lambda f: f.is_integer(), res.x))):
                if self.LOWER_BOUND < -res.fun:
                    self.LOWER_BOUND = -res.fun

                if self.opt_val is None or self.opt_val < -res.fun:
                    self.opt_val = -res.fun
                    self.opt_x = res.x

                continue

            # 进行分枝
            else:
                # 寻找 x 中第一个不是整数的，取其下标 idx
                idx = 0
                for i, x in enumerate(res.x):
                    if not x.is_integer():
                        break
                    idx += 1

                # 构建新的约束条件（分割
                new_con1 = np.zeros(A_ub.shape[1])
                new_con1[idx] = -1
                new_con2 = np.zeros(A_ub.shape[1])
                new_con2[idx] = 1
                new_A_ub1 = np.insert(A_ub, A_ub.shape[0], new_con1, axis=0)
                new_A_ub2 = np.insert(A_ub, A_ub.shape[0], new_con2, axis=0)
                new_b_ub1 = np.insert(
                    b_ub, b_ub.shape[0], -math.ceil(res.x[idx]), axis=0
                )
                new_b_ub2 = np.insert(
                    b_ub, b_ub.shape[0], math.floor(res.x[idx]), axis=0
                )

                # 将新约束条件加入队列，先加最优值大的那一支
                r1 = linprog(
                    self.c, new_A_ub1, new_b_ub1, self.A_eq, self.b_eq, self.bounds
                )
                r2 = linprog(
                    self.c, new_A_ub2, new_b_ub2, self.A_eq, self.b_eq, self.bounds
                )
                if not r1.success and r2.success:
                    self.Q.put((r2, new_A_ub2, new_b_ub2))
                elif not r2.success and r1.success:
                    self.Q.put((r1, new_A_ub1, new_b_ub1))
                elif r1.success and r2.success:
                    if -r1.fun > -r2.fun:
                        self.Q.put((r1, new_A_ub1, new_b_ub1))
                        self.Q.put((r2, new_A_ub2, new_b_ub2))
                    else:
                        self.Q.put((r2, new_A_ub2, new_b_ub2))
                        self.Q.put((r1, new_A_ub1, new_b_ub1))


def test1():
    """此测试的真实最优解为 [4, 2]"""
    c = np.array([40, 90])
    A = np.array([[9, 7], [7, 20]])
    b = np.array([56, 70])
    Aeq = None
    beq = None
    bounds = [(0, None), (0, None)]

    solver = ILP(c, A, b, Aeq, beq, bounds)
    solver.solve()

    print("Test 1's result:", solver.opt_val, solver.opt_x)
    print("Test 1's true optimal x: [4, 2]\n")


def test2():
    """此测试的真实最优解为 [2, 4]"""
    c = np.array([3, 13])
    A = np.array([[2, 9], [11, -8]])
    b = np.array([40, 82])
    Aeq = None
    beq = None
    bounds = [(0, None), (0, None)]

    solver = ILP(c, A, b, Aeq, beq, bounds)
    solver.solve()

    print("Test 2's result:", solver.opt_val, solver.opt_x)
    print("Test 2's true optimal x: [2, 4]\n")


if __name__ == "__main__":
    test1()
    test2()

Test 1's result: 340.0 [4. 2.]
Test 1's true optimal x: [4, 2]

Test 2's result: 58.0 [2. 4.]
Test 2's true optimal x: [2, 4]



In [ ]:
import pulp as pp

# 定义一个问题，第一个参数就是一个名字，随便取，后面是一个int型，说明的是求最大或者最小
# pp.LpMinimize其实就是一个int，值为1
# pp.LpMaximize也是一个int，值为-1.
# 在参数后面也可以直接写1或者-1表示
# 其实默认的都是求最小值，在求最大值的时候两边都要乘以-1，转成最小值求解。
# 						所以最小值参数为1，最大值参数为-1
mylp = pp.LpProblem("lp1", pp.LpMinimize)

# 定义未知数，标记取值范围，cat为限制条件，Integer表示整数型，Continuous表示连续型
x1 = pp.LpVariable("x1", lowBound=0, cat="Integer")
x2 = pp.LpVariable("x2", lowBound=0, cat="Integer")
x3 = pp.LpVariable("x3", lowBound=0, cat="Integer")

# 在pulp中用+=符号，加约束和目标函数
# 只支持 = ，>= ， <= 不支持> , <
mylp += 3 * x1 + 4 * x2 + x3
mylp += x1 + 6 * x2 + 2 * x3 >= 5
mylp += 2 * x1 >= 3

In [2]:
mylp

lp1:
MINIMIZE
3*x1 + 4*x2 + 1*x3 + 0
SUBJECT TO
_C1: x1 + 6 x2 + 2 x3 >= 5

_C2: 2 x1 >= 3

VARIABLES
0 <= x1 Integer
0 <= x2 Integer
0 <= x3 Integer

In [3]:
# 最后一定要调用这个函数。才能得到解
# 他返回一个int型，是一个status，表示的是pp.LpStatus里面对应的数字含义
# {0: 'Not Solved',
#   1: 'Optimal',最佳的
#  -1: 'Infeasible',
#  -2: 'Unbounded',
#  -3: 'Undefined'}

i = mylp.solve()
print(i)  # 结果是1，返回1就说明成功计算了

1


In [4]:
# 得到这些值
[v.varValue for v in mylp.variables()]

[2.0, 0.0, 2.0]

In [6]:
print(mylp.objective)

pp.value(mylp.objective)

3*x1 + 4*x2 + x3


8.0

In [1]:
# 导入sympy包，用于求导，方程组求解等等
from sympy import *

# 设置变量
x1 = symbols("x1")
x2 = symbols("x2")
alpha = symbols("alpha")
# beta = symbols("beta")

# 构造拉格朗日等式
L = 60 - 10 * x1 - 4 * x2 + x1 * x1 + x2 * x2 - x1 * x2 - alpha * (x1 + x2 - 8)

# 求导，构造KKT条件
difyL_x1 = diff(L, x1)  # 对变量x1求导
difyL_x2 = diff(L, x2)  # 对变量x2求导
difyL_alpha = diff(L, alpha)  # 对alpha求导
# 求解KKT等式
aa = solve([difyL_x1, difyL_x2, difyL_alpha], [x1, x2, alpha])
print(aa)

{x1: 5, x2: 3, alpha: -3}


In [3]:
difyL_x1

-alpha + 2*x1 - x2 - 10

: 

In [10]:
from scipy.optimize import minimize
import numpy as np


# 目标函数：
def func(args):
    fun = lambda x: 60 - 10 * x[0] - 4 * x[1] + x[0] ** 2 + x[1] ** 2 - x[0] * x[1]
    return fun


# 约束条件，包括等式约束和不等式约束
def con(args):
    cons = {"type": "eq", "fun": lambda x: x[0] + x[1] - 8}
    return cons


if __name__ == "__main__":
    args = ()
    args1 = ()
    cons = con(args1)
    x0 = np.array((2.0, 1.0))  # 设置初始值，初始值的设置很重要，很容易收敛到另外的极值点中，建议多试几个值

    # 求解#
    res = minimize(func(args), x0, method="SLSQP", constraints=cons)
    print(res)

 message: Optimization terminated successfully
 success: True
  status: 0
     fun: 17.000000000000007
       x: [ 5.000e+00  3.000e+00]
     nit: 4
     jac: [-3.000e+00 -3.000e+00]
    nfev: 12
    njev: 4


In [11]:
from scipy.optimize import minimize
import numpy as np

# 计算（2+x1）/ （1+x2) - 3x1 + 4x3的最小值，其中x1、x2、x3范围在0.1到0.9之间


def fun(args):
    a, b, c, d = args
    v = lambda x: (a + x[0]) / (b + x[1]) - c * x[0] + d * x[2]
    return v


def con(args):
    # 约束条件 分为eq 和ineq
    # eq表示 函数结果等于0 ； ineq 表示 表达式大于等于0
    x1min, x1max, x2min, x2max, x3min, x3max = args
    cons = (
        {"type": "ineq", "fun": lambda x: x[0] - x1min},
        {"type": "ineq", "fun": lambda x: -x[0] + x1max},
        {"type": "ineq", "fun": lambda x: x[1] - x2min},
        {"type": "ineq", "fun": lambda x: -x[1] + x2max},
        {"type": "ineq", "fun": lambda x: x[2] - x3min},
        {"type": "ineq", "fun": lambda x: -x[2] + x3max},
    )
    return cons


if __name__ == "__main__":
    # 定义常量值
    args = (2, 1, 3, 4)  # a,b,c,d
    # 设置参数范围/约束条件
    args1 = (0.1, 0.9, 0.1, 0.9, 0.1, 0.9)  # x1min, x1max, x2min, x2max
    cons = con(args1)
    # 设置初始猜测值
    x0 = np.asarray((0.5, 0.5, 0.5))
    res = minimize(fun(args), x0, method="SLSQP", constraints=cons)
    print(res)

 message: Optimization terminated successfully
 success: True
  status: 0
     fun: -0.773684210526435
       x: [ 9.000e-01  9.000e-01  1.000e-01]
     nit: 2
     jac: [-2.474e+00 -8.033e-01  4.000e+00]
    nfev: 8
    njev: 2
